[Node 18: Synchronisation](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node18.html)

Navigation:

 **Next:** [Python Threads und GIL](node19.ipynb) **Up:** [Threads and Multi–Processing](node16.ipynb) **Previous:** [Die Thread Klasse](node17.ipynb)

##  Synchronisation
 In den bisherigen Beispielen laufen die Threads unabhängig  voneinander. Komplizierter wird es wenn sie auf gemeinsame Datenbereiche zugreifen, insbesondere wenn ein Thread schreibt und ein anderes liest.   Das folgende –etwas konstruierte– Beispiel illustriert das Problem  

---

In [ ]:
import threading
import time
class CounterThread( threading.Thread ) :
    """
    A sample thread class
    """
    
    def __init__(self, co, name="TestThread"):
        """
        Constructor, setting initial variables
        """
        self.co = co
        threading.Thread.__init__(self, name=name)
        
    def run(self):
        """
        overload of threading.thread.run()
        main control loop
        """
        print ("%s starts" % (self.getName(),))

        self.co.count()

    pass

class Counter(object):
    def __init__(self):
        self.num = 0
    def count(self):
        limit = self.num + 100;
        while self.num != limit:
            print (self.num)
            self.num += 1
            time.sleep(0.0001) # sleep 0.1 ms
    pass
# main
c = Counter()
ct1 = CounterThread(c,"ct1")
ct2 = CounterThread(c,"ct2")

ct1.start()
ct2.start()

# Note: even when stopping kernel, threads will continue to run in background


 Beide Threads greifen auf dasselbe   <font color=#008000> *Counter–Objekt*</font>  zu, d.h. sie benutzen diesselbe   <font color=#ff0000> **Member-variable**</font>  **self.num**.  
 
Abhängig vom zufälligen Ablauf endet das Programm vernünftig oder geht in eine Quasi–Endlosschleife. Es ist nicht–deterministisch, obwohl keine Zufallszahlen benutzt  werden... 

 <font color=#ff0000> **Probieren Sie's aus !**</font>  
 
Man kann im   <font color=#008000> *CounterThread*</font>-Beispiel die Variablen anders definieren/einsetzen und das spezifische Problem damit beheben, z.B.   <font color=#008000> *self.num*</font>  als lokale Variable in   <font color=#008000> *count()*</font>  Methode.   

Das ist aber keine Lösung für den allgemeinen Fall.   

Echte Abhilfe bietet in Python (u.a.) der    <font color=#ff0000> **Locking**</font>  Mechanismus:  
*  <font color=#0000e6> ``lck=threading.Lock()``</font>  Objekt wird erzeugt 
* Aufruf von   <font color=#0000e6> ``lck.acquire()``</font>  bevor kritischer Bereich ausgeführt wird 
* 1. Thread der   <font color=#0000e6> ``lck.acquire()``</font>  ruft läuft weiter 
* Nächster Thread, der   <font color=#0000e6> ``lck.acquire()``</font>  muss warten  
* bis 1. Thread   <font color=#0000e6> ``lck.release()``</font>  ruft 
* usw.  
Damit wird sichergestellt, dass Methode   <font color=#008000> *count()*</font>  nur von   <font color=#ff0000> **einem**</font>  Thread benutzt wird, alle anderen sind solange blockiert.  

---

In [ ]:
import threading
import time
class CounterThread( threading.Thread ) :
    """
    A sample thread class
    """
    
    def __init__(self, co, name="TestThread"):
        """
        Constructor, setting initial variables
        """
        self.co = co
        threading.Thread.__init__(self, name=name)
        
    def run(self):
        """
        overload of threading.thread.run()
        main control loop
        """
        print ("%s starts" % (self.getName(),))

        self.co.count()

    pass

class Counter(object):
    lck = threading.Lock()
    def __init__(self):
        self.num = 0
    def count(self):
        self.lck.acquire() # get lock, waits until it's free
        limit = self.num + 100;
        while self.num != limit:
            print (self.num)
            self.num += 1
            time.sleep(0.0001) # sleep 0.1 ms

        self.lck.release() # release lock
    pass

# main
c = Counter()
ct1 = CounterThread(c,"ct1")
ct2 = CounterThread(c,"ct2")
ct1.start()
ct2.start()
